In [ ]:
!pip install pykalman

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from prophet import Prophet
from statsmodels.tsa.seasonal import STL
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
#from missingpy import MissForest



In [ ]:
#dataset_path = '/content/drive/MyDrive/eksik_veri_simulasyonlari/MS_missing_with_existing_25pct.xlsx'
dataset_path = '/content/drive/MyDrive/VNS_dataset/VNS_Tepebasi.xlsx'
#dataset_path = '/content/drive/MyDrive/simulated_missing_xlsx/C.xlsx'

df = pd.read_excel(dataset_path, parse_dates=['Tarih'])
df.replace("-", np.nan, inplace=True)
df['Tarih'] = pd.to_datetime(df['Tarih'], dayfirst=True, errors='coerce')

df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].apply(lambda x: np.nan if x < 0 else x)
max_val = np.nanmax(df['PM10 ( µg/m3 )'])
df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].clip(lower=0, upper=max_val)
df_new = df.iloc[29530:]
print(df_new.head())
#series = df['PM 2.5 ( µg/m3 )'].copy()
series = df['PM10 ( µg/m3 )']
#resid_full = series
#dates = df['Tarih'].copy()
#series=series.iloc[35049:]
#dates = dates.iloc[35049:]
#print(dates.head())
n_total = len(series)
split_idx = int(0.8 * n_total)
train_series = series[:split_idx]
test_series = series[split_idx:]
test_dates = dates[split_idx:]
train_series = train_series.reset_index(drop=True)
subsample_dates_train = dates[:split_idx].reset_index(drop=True)
print(len(series), len(train_series), len(test_series))

In [ ]:
import time
import pandas as pd
import numpy as np
from pykalman import KalmanFilter
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

summary_records = []
prediction_dict = {}

for run in range(1, 11):
    print(f"▶️ Iteration {run} başlıyor...")
    dataset_path = '/content/drive/MyDrive/eksik_veri_simulasyonlari/Tepebasi_missing_with_existing_10pct.xlsx'
    #dataset_path = '/content/drive/MyDrive/VNS_dataset/VNS_Odunpazari.xlsx'
    df = pd.read_excel(dataset_path, parse_dates=['Tarih'])
    df.replace("-", np.nan, inplace=True)
    df = df.iloc[35048:]
    print(df.head())
    df['Tarih'] = pd.to_datetime(df['Tarih'], dayfirst=True, errors='coerce')
    df['PM 2.5 ( µg/m3 )'] = df['PM 2.5 ( µg/m3 )'].apply(lambda x: np.nan if x < 0 else x)
    df['PM 2.5 ( µg/m3 )'] = df['PM 2.5 ( µg/m3 )'].clip(lower=0, upper=np.nanmax(df['PM 2.5 ( µg/m3 )']))

    series = df['PM 2.5 ( µg/m3 )']
    dates = df['Tarih']
    #series = series.iloc[18630:].reset_index(drop=True)
    #dates = dates.iloc[18630:].reset_index(drop=True)
    print(dates.head())
    n_total = len(series)
    original_nan_mask = series.isna().to_numpy()
    split_idx = int(0.8 * n_total)
    train_series = series[:split_idx].reset_index(drop=True)
    test_series = series[split_idx:].reset_index(drop=True)

    # === 1. IMPUTATION YÖNTEMLERİ ===
    def modified_smape(y_true, y_pred):
      return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))

    def kalman_filter_imputation(train, test):
        kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1,
                          initial_state_covariance=1, transition_matrices=[1],
                          observation_matrices=[1], transition_covariance=1,
                          observation_covariance=1)
        train_masked = np.ma.array(train, mask=np.isnan(train))
        train_imputed, _ = kf.smooth(train_masked)

        combined = np.concatenate([train_imputed.flatten(), test])
        combined_masked = np.ma.array(combined, mask=np.isnan(combined))
        combined_imputed, _ = kf.smooth(combined_masked)
        test_imputed = combined_imputed[len(train):]
        return train_imputed.flatten(), test_imputed.flatten()

    def interpolation_imputation(train, test):
        train_imputed = pd.Series(train).interpolate(limit_direction='both').to_numpy()
        combined = np.concatenate([train_imputed, test])
        combined_imputed = pd.Series(combined).interpolate(limit_direction='both').to_numpy()
        return train_imputed, combined_imputed[len(train):]

    def backward_fill_imputation(train, test):
        return pd.Series(train).bfill().ffill().to_numpy(), pd.Series(test).bfill().ffill().to_numpy()

    def svr_imputation_separate(train, test, n_lags=3):
        def create_lagged_df(series):
            df = pd.DataFrame({'y': series})
            for i in range(1, n_lags + 1):
                df[f'lag_{i}'] = df['y'].shift(i)
            return df

        feature_names = [f'lag_{i}' for i in range(1, n_lags + 1)]
        train_df = create_lagged_df(pd.Series(train)).dropna()
        X_train = train_df[feature_names].to_numpy()
        y_train = train_df['y'].to_numpy()
        svr = SVR().fit(X_train, y_train)

        train_filled = np.copy(train)
        for idx in np.where(np.isnan(train))[0]:
            if idx >= n_lags and not np.isnan(train[idx - n_lags:idx]).any():
                x_pred = train[idx - n_lags:idx][::-1].reshape(1, -1)
                train_filled[idx] = svr.predict(x_pred)[0]
        train_filled = pd.Series(train_filled).interpolate().bfill().ffill().to_numpy()

        test_filled = np.copy(test)
        for idx in np.where(np.isnan(test))[0]:
            full_input = np.concatenate([train_filled[-n_lags:], test[:idx]])
            if len(full_input) >= n_lags and not np.isnan(full_input[-n_lags:]).any():
                x_pred = full_input[-n_lags:][::-1].reshape(1, -1)
                test_filled[idx] = svr.predict(x_pred)[0]
        test_filled = pd.Series(test_filled).interpolate().bfill().ffill().to_numpy()
        return train_filled, test_filled


    start_time = time.time()
    # === 2. TÜM YÖNTEMLERİ UYGULA ===
    kalman_train, kalman_test = kalman_filter_imputation(train_series.to_numpy(), test_series.to_numpy())
    linear_train, linear_test = interpolation_imputation(train_series, test_series)
    bfill_train, bfill_test = backward_fill_imputation(train_series, test_series)
    svr_train, svr_test = svr_imputation_separate(train_series.to_numpy(), test_series.to_numpy())

    # === 3. STACKING VERİSİ (TÜM DOLU GÖZLEMLER) ===
    non_nan_train_idx = np.where(~np.isnan(train_series))[0]

    X_train_stack = np.column_stack([
        kalman_train[non_nan_train_idx],
        linear_train[non_nan_train_idx],
        bfill_train[non_nan_train_idx],
        svr_train[non_nan_train_idx]
    ])
    y_train_stack = train_series.to_numpy()[non_nan_train_idx]

    nan_train_idx = np.where(np.isnan(train_series))[0]
    nan_test_idx = np.where(np.isnan(test_series))[0]

    X_test_stack = np.column_stack([
        kalman_test[nan_test_idx],
        linear_test[nan_test_idx],
        bfill_test[nan_test_idx],
        svr_test[nan_test_idx]
    ])

    # === 4. META MODEL: GRADIENT BOOSTING ===
    if len(X_train_stack) >= 10:
        meta_model = GradientBoostingRegressor()
        meta_model.fit(X_train_stack, y_train_stack)

        # Yalnızca eksik değerler doldurulacak
        train_series.iloc[nan_train_idx] = meta_model.predict(
            np.column_stack([
                kalman_train[nan_train_idx],
                linear_train[nan_train_idx],
                bfill_train[nan_train_idx],
                svr_train[nan_train_idx]
            ])
        )
        test_series.iloc[nan_test_idx] = meta_model.predict(X_test_stack)
    else:
        print("⚠️ Yetersiz eğitim verisi. Kalman ile dolduruluyor.")
        train_series.iloc[nan_train_idx] = kalman_train[nan_train_idx]
        test_series.iloc[nan_test_idx] = kalman_test[nan_test_idx]

    filled_series = pd.concat([train_series, test_series], ignore_index=True)
    end_time = time.time()
    imputation_time = end_time - start_time
    print(f"Imputation Time: {imputation_time} seconds")

    # === 5. GÖRSELLEŞTİRME ===
    plt.figure(figsize=(12, 6))
    plt.plot(dates, series, label='Orijinal Seri (Eksik)', alpha=0.6)
    plt.plot(dates, filled_series, label='Stacking + GB Regressor ile Doldurulmuş Seri')
    plt.xlabel('Tarih')
    plt.ylabel('PM 2.5 ( µg/m3 )')
    plt.title('Eksik Veri Doldurma: Stacking ve GB Regressor')
    plt.legend()
    plt.grid(True)
    plt.show()

    print("Eksik değer sayısı (doldurma sonrası):", filled_series.isnull().sum())
    filled_only = filled_series
    #filled_only = final_filled_series
    print(len(filled_only))

    # === 1. Train / Test böl
    split_ratio = 0.8
    split_idx = int(len(filled_only) * split_ratio)
    train_series = filled_only[:split_idx]
    test_series = filled_only[split_idx:]
    print(len(train_series))
    print(len(test_series))
    # === 2. Normalizasyon
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_series.to_numpy().reshape(-1, 1))
    test_scaled = scaler.transform(test_series.to_numpy().reshape(-1, 1))

    # === 3. Sliding window veri hazırlığı
    def create_lstm_dataset(series, look_back=24):
        X, y = [], []
        for i in range(len(series) - look_back):
            X.append(series[i:i+look_back])
            y.append(series[i+look_back])
        return np.array(X), np.array(y)

    look_back = 24
    X_train, y_train = create_lstm_dataset(train_scaled, look_back)
    X_test, y_test = create_lstm_dataset(test_scaled, look_back)

    # LSTM input boyutu: (samples, timesteps, features)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    train_time_start = time.time()
    # === 4. LSTM Modeli
    model = Sequential()
    model.add(LSTM(30, input_shape=(look_back, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.summary()

    # === 5. Model eğitimi
    model.fit(X_train, y_train, epochs=100, batch_size=50, verbose=1)
    train_time_end = time.time()
    train_time = train_time_end - train_time_start
    print(f"Training Time: {train_time} seconds")

    # === 6. Tahmin ve ters dönüşüm
    train_pred = model.predict(X_train)

    test_time_start = time.time()
    test_pred = model.predict(X_test)
    test_time_end = time.time()

    test_time = test_time_end - test_time_start
    print(f"Test Time: {test_time} seconds")

    trainPredict = scaler.inverse_transform(train_pred)
    trainY = scaler.inverse_transform(y_train.reshape(-1, 1))
    testPredict = scaler.inverse_transform(test_pred)
    testY = scaler.inverse_transform(y_test.reshape(-1, 1))
    print(len(testY), len(testPredict))

    # Orijinal test verisi (eksik değer içeren)
    #orig_train = alldata[0:train_size]  # Test kısmının orijinali
    #orig_mask_train = ~np.isnan(orig_train)  # Eksik olmayanlar için maske
    #valid_mask_train = orig_mask_train[look_back+1:]  # look_back sonrası geçerli indeksler


    orig_train = series[0:split_idx]
    orig_mask_train = ~np.isnan(orig_train)
    valid_mask_train = orig_mask_train[look_back:look_back + len(trainY)]

    print(f"valid_mask_train: {valid_mask_train.shape}")
    print(f"trainY: {trainY.shape}, trainPredict: {trainPredict.shape}")

    # Maskeye göre filtreleme
    trainY = trainY.ravel()
    trainPredict = trainPredict.ravel()
    filtered_trainY = trainY[valid_mask_train]
    filtered_trainPredict = trainPredict[valid_mask_train]

    trainMAE = mean_absolute_error(filtered_trainY, filtered_trainPredict)
    trainMSE = np.sqrt(mean_squared_error(trainY, trainPredict))
    trainmsMAPE = modified_smape(trainY, trainPredict)
    print("train_MAE:", trainMAE)
    print("train_MSE:", trainMSE)
    print("train_modifiedsMAPE:", trainmsMAPE)


    # Orijinal test verisi (eksik değer içeren)
    #orig_test = alldata[train_size+1:]  # Test kısmının orijinali
    #orig_mask = ~np.isnan(orig_test)  # Eksik olmayanlar için maske
    #valid_mask = orig_mask[look_back:]  # look_back sonrası geçerli indeksler

    orig_test = series[split_idx+look_back:]  # Güncellendi
    orig_mask = ~np.isnan(orig_test)
    valid_mask = orig_mask  # look_back sonrası eksik olmayan veriler

    print("TestPredict shape:", testPredict.shape)
    testY = testY.flatten()
    testPredict = testPredict.flatten()

    # Maskeye göre filtreleme
    filtered_testY = testY[valid_mask]
    filtered_testPredict = testPredict[valid_mask]

    print("Filtered TestY shape:", filtered_testY.shape)
    print("Filtered TestPredict shape:", filtered_testPredict.shape)

    if len(filtered_testY) == 0:
        print("Hata: Filtrelenmiş test verileri boş.")
    else:
        testMAE = mean_absolute_error(filtered_testY, filtered_testPredict)
        testMSE = np.sqrt(mean_squared_error(filtered_testY, filtered_testPredict))
        testmsMAPE = modified_smape(filtered_testY, filtered_testPredict)
        print("test_MAE:", testMAE)
        print("test_MSE:", testMSE)
        print("test_modifiedsMAPE:", testmsMAPE)

    '''
    # === 8. Grafik
    plt.figure(figsize=(15, 5))
    plt.plot(y_true_filtered, label="Gerçek", linewidth=2)
    plt.plot(y_pred_filtered, label="Tahmin", linewidth=2, alpha=0.7)
    plt.title("LSTM Test Tahmin Performansı")
    plt.legend()
    plt.grid(True)
    plt.show()
    '''
    summary_records.append({
        "Iteration": run,
        "Imputation Time": imputation_time,
        "Train MAE": trainMAE,
        "Train RMSE": trainMSE,
        "Train msMAPE": trainmsMAPE,
        "Test MAE": testMAE,
        "Test RMSE": testMSE,
        "Test msMAPE": testmsMAPE
    })

    # Tahmin değerlerini sakla (her iterasyon için ayrı sütun)
    prediction_dict[f"y_true_{run}"] = filtered_testY
    prediction_dict[f"y_pred_{run}"] = filtered_testPredict

# ——— SONUÇLARI KAYDETMEK İÇİN DATAFRAME OLUŞTUR ———
summary_df = pd.DataFrame(summary_records)
summary_df = summary_df.round({
    "Imputation Time": 4,
    "Train MSE": 6, "Train MAE": 6, "Train RMSE": 6, "Train msMAPE": 4,
    "Test MSE": 6, "Test MAE": 6, "Test RMSE": 6, "Test msMAPE": 4
})
summary_df.to_excel("Metin_sonmez_imputation_lstm_summary.xlsx", index=False)


# Tüm tahminleri hizalamak için en kısa uzunluğu belirle
min_len = min(len(v) for v in prediction_dict.values())
prediction_df = pd.DataFrame({k: v[:min_len] for k, v in prediction_dict.items()})

print(prediction_df.head())
# Excel’e kaydet (çalıştırmak istersen aşağıdaki yorumları kaldır)
# summary_df.to_excel("imputation_lstm_summary.xlsx", index=False)
prediction_df.to_excel("NewOdunpazari_imputation_lstm_predictions.xlsx", index=False)

print("✅ 10 çalıştırmalık özet ve tahmin sonuçları hazır.")


In [ ]:
import itertools
import time
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

def find_best_arima(train_data, validation_size=0.2, p_range=(0, 3), d_range=(0, 2), q_range=(0, 3), metric="rmse"):
    """
    Train verisi için en iyi (p, d, q) ARIMA parametrelerini seçer.

    - Eğer metric="rmse" ise, validation için train_data'yı ikiye böler ve bölünen train set ile modeli eğitir, doğrulama seti üzerinde RMSE hesaplar.
    - Eğer metric="aic" ise, tüm train_data ile modeli eğitir ve AIC'yi hesaplar.
    """

    best_score = float("inf")  # En iyi RMSE veya AIC değerini saklar
    best_params = None
    best_model = None

    # RMSE için train seti ayırma (validation kullanılır)
    if metric == "rmse":
        split_index = int(len(train_data) * (1 - validation_size))
        train_set, val_set = train_data[:split_index], train_data[split_index:]
    elif metric=="aic":
        train_set = train_data  # AIC için tüm veri kullanılır
        val_set = None  # AIC için validation set yok

    # Grid Search için tüm (p, d, q) kombinasyonlarını oluştur
    pdq_combinations = list(itertools.product(range(*p_range), range(*d_range), range(*q_range)))

    for pdq in pdq_combinations:
        try:
            if metric == "rmse":
                model = ARIMA(train_set, order=pdq)  # RMSE için sadece train_set ile eğit
            else:
                model = ARIMA(train_data, order=pdq)  # AIC için tüm train_data kullanılır

            model_fit = model.fit()

            # RMSE ile değerlendirme
            if metric == "rmse":
                predictions = model_fit.forecast(steps=len(val_set))
                rmse = np.sqrt(mean_squared_error(val_set, predictions))
                score = rmse
                print(f"ARIMA{pdq} - RMSE: {rmse}")

            # AIC ile değerlendirme
            elif metric == "aic":
                aic = model_fit.aic
                score = aic
                print(f"ARIMA{pdq} - AIC: {aic}")

            else:
                raise ValueError("metric parametresi 'rmse' veya 'aic' olmalıdır.")

            # En iyi modeli güncelle
            if score < best_score:
                best_score, best_params = score, pdq
                best_model = model_fit

        except Exception as e:
            print(f"ARIMA{pdq} başarısız: {e}")
            continue

    return best_params


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
def svr_grid_search(X,y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

      # Hiperparametre arama aralıkları
    param_grid = {
    'C': [1, 10, 100],  # Çok küçük ve çok büyük C değerlerini çıkardık
    'epsilon': [0.01, 0.1],  # Yüksek epsilon değerleri genellikle tahmin doğruluğunu düşürür
    'kernel': ['rbf'],  # Sadece RBF kernel kullanıyoruz
    'gamma': ['scale', 0.1]  # Sadece "scale" ve makul bir sabit değer kullanıyoruz
    }

    # SVR Modeli
    svr = SVR()

    # Grid Search CV
    grid_search = GridSearchCV(svr, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)

    # En iyi model ve hiperparametreler
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    print("En iyi parametreler:", best_params)
    return  best_params

In [ ]:
#STACKING+ARIMA
import time
import pandas as pd
import numpy as np
from pykalman import KalmanFilter
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import itertools

summary_records = []
prediction_dict = {}

for run in range(1, 11):
    print(f"▶️ Iteration {run} başlıyor...")
    dataset_path = '/content/drive/MyDrive/eksik_veri_simulasyonlari/Tepebasi_missing_with_existing_10pct.xlsx'
    #dataset_path = '/content/drive/MyDrive/VNS_dataset/VNS_Cumhuriyet_Bulvari.xlsx'
    df = pd.read_excel(dataset_path, parse_dates=['Tarih'])
    df.replace("-", np.nan, inplace=True)
    #df = df.iloc[35049:]
    print(df.head())
    df['Tarih'] = pd.to_datetime(df['Tarih'], dayfirst=True, errors='coerce')
    df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].apply(lambda x: np.nan if x < 0 else x)
    df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].clip(lower=0, upper=np.nanmax(df['PM10 ( µg/m3 )']))

    series = df['PM10 ( µg/m3 )']
    dates = df['Tarih']
    #series = series.iloc[18630:].reset_index(drop=True)
    #dates = dates.iloc[18630:].reset_index(drop=True)
    print(dates.head())
    n_total = len(series)
    split_idx = int(0.8 * n_total)
    train_series = series[:split_idx].reset_index(drop=True)
    test_series = series[split_idx:].reset_index(drop=True)
    # Eğitim (Train) ve Test bölümleri
    train_data = series[:split_idx]  # İlk %80
    test_data =series[split_idx:]   # Son %20
    orig_mask = ~np.isnan(test_data)
    orig_mask_train = ~np.isnan(train_data)
    # === 1. IMPUTATION YÖNTEMLERİ ===

    def kalman_filter_imputation(train, test):
        kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1,
                          initial_state_covariance=1, transition_matrices=[1],
                          observation_matrices=[1], transition_covariance=1,
                          observation_covariance=1)
        train_masked = np.ma.array(train, mask=np.isnan(train))
        train_imputed, _ = kf.smooth(train_masked)

        combined = np.concatenate([train_imputed.flatten(), test])
        combined_masked = np.ma.array(combined, mask=np.isnan(combined))
        combined_imputed, _ = kf.smooth(combined_masked)
        test_imputed = combined_imputed[len(train):]
        return train_imputed.flatten(), test_imputed.flatten()

    def interpolation_imputation(train, test):
        train_imputed = pd.Series(train).interpolate(limit_direction='both').to_numpy()
        combined = np.concatenate([train_imputed, test])
        combined_imputed = pd.Series(combined).interpolate(limit_direction='both').to_numpy()
        return train_imputed, combined_imputed[len(train):]

    def backward_fill_imputation(train, test):
        return pd.Series(train).bfill().ffill().to_numpy(), pd.Series(test).bfill().ffill().to_numpy()

    def svr_imputation_separate(train, test, n_lags=3):
        def create_lagged_df(series):
            df = pd.DataFrame({'y': series})
            for i in range(1, n_lags + 1):
                df[f'lag_{i}'] = df['y'].shift(i)
            return df

        feature_names = [f'lag_{i}' for i in range(1, n_lags + 1)]
        train_df = create_lagged_df(pd.Series(train)).dropna()
        X_train = train_df[feature_names].to_numpy()
        y_train = train_df['y'].to_numpy()
        svr = SVR().fit(X_train, y_train)

        train_filled = np.copy(train)
        for idx in np.where(np.isnan(train))[0]:
            if idx >= n_lags and not np.isnan(train[idx - n_lags:idx]).any():
                x_pred = train[idx - n_lags:idx][::-1].reshape(1, -1)
                train_filled[idx] = svr.predict(x_pred)[0]
        train_filled = pd.Series(train_filled).interpolate().bfill().ffill().to_numpy()

        test_filled = np.copy(test)
        for idx in np.where(np.isnan(test))[0]:
            full_input = np.concatenate([train_filled[-n_lags:], test[:idx]])
            if len(full_input) >= n_lags and not np.isnan(full_input[-n_lags:]).any():
                x_pred = full_input[-n_lags:][::-1].reshape(1, -1)
                test_filled[idx] = svr.predict(x_pred)[0]
        test_filled = pd.Series(test_filled).interpolate().bfill().ffill().to_numpy()
        return train_filled, test_filled


    start_time = time.time()
    # === 2. TÜM YÖNTEMLERİ UYGULA ===
    kalman_train, kalman_test = kalman_filter_imputation(train_series.to_numpy(), test_series.to_numpy())
    linear_train, linear_test = interpolation_imputation(train_series, test_series)
    bfill_train, bfill_test = backward_fill_imputation(train_series, test_series)
    svr_train, svr_test = svr_imputation_separate(train_series.to_numpy(), test_series.to_numpy())

    # === 3. STACKING VERİSİ (TÜM DOLU GÖZLEMLER) ===
    non_nan_train_idx = np.where(~np.isnan(train_series))[0]

    X_train_stack = np.column_stack([
        kalman_train[non_nan_train_idx],
        linear_train[non_nan_train_idx],
        bfill_train[non_nan_train_idx],
        svr_train[non_nan_train_idx]
    ])
    y_train_stack = train_series.to_numpy()[non_nan_train_idx]

    nan_train_idx = np.where(np.isnan(train_series))[0]
    nan_test_idx = np.where(np.isnan(test_series))[0]

    X_test_stack = np.column_stack([
        kalman_test[nan_test_idx],
        linear_test[nan_test_idx],
        bfill_test[nan_test_idx],
        svr_test[nan_test_idx]
    ])

    # === 4. META MODEL: GRADIENT BOOSTING ===
    if len(X_train_stack) >= 10:
        meta_model = GradientBoostingRegressor(alpha=0.8,max_depth=4)
        meta_model.fit(X_train_stack, y_train_stack)

        # Yalnızca eksik değerler doldurulacak
        train_series.iloc[nan_train_idx] = meta_model.predict(
            np.column_stack([
                kalman_train[nan_train_idx],
                linear_train[nan_train_idx],
                bfill_train[nan_train_idx],
                svr_train[nan_train_idx]
            ])
        )
        test_series.iloc[nan_test_idx] = meta_model.predict(X_test_stack)
    else:
        print("⚠️ Yetersiz eğitim verisi. Kalman ile dolduruluyor.")
        train_series.iloc[nan_train_idx] = kalman_train[nan_train_idx]
        test_series.iloc[nan_test_idx] = kalman_test[nan_test_idx]
    print("doldurmasonrasri:", len(test_series))

    filled_series = pd.concat([train_series, test_series], ignore_index=True)
    end_time = time.time()
    imputation_time = end_time - start_time
    print(f"Imputation Time: {imputation_time} seconds")

    # === 5. GÖRSELLEŞTİRME ===
    plt.figure(figsize=(12, 6))
    plt.plot(dates, series, label='Orijinal Seri (Eksik)', alpha=0.6)
    plt.plot(dates, filled_series, label='Stacking + GB Regressor ile Doldurulmuş Seri')
    plt.xlabel('Tarih')
    plt.ylabel('PM 2.5 ( µg/m3 )')
    plt.title('Eksik Veri Doldurma: Stacking ve GB Regressor')
    plt.legend()
    plt.grid(True)
    plt.show()

    print("Eksik değer sayısı (doldurma sonrası):", filled_series.isnull().sum())
    filled_only = filled_series
    #filled_only = final_filled_series
    print(len(filled_only))

    # === 1. Train / Test böl
    split_ratio = 0.8
    split_idx = int(len(filled_only) * split_ratio)
    train_series = filled_only[:split_idx]
    test_series = filled_only[split_idx:]
    print(len(train_series))
    print("doldurmasonrasi bolme:", len(test_series))
    imputed_train = train_series
    imputed_test = test_series


    best_params= find_best_arima(imputed_train)
    best_model = ARIMA(imputed_train, order=best_params)
    training_time_start=time.time()
    best_model_fit=best_model.fit()

    training_time_end=time.time()
    training_time=training_time_end-training_time_start
    test_start = time.time()
    testPredict = best_model_fit.forecast(steps=len(imputed_test))
    test_end = time.time()
    trainPredict = best_model_fit.predict(start=0, end=len(imputed_train)-1)
    print("tahmin", len(testPredict))

    filtered_trainY = imputed_train.reset_index(drop=True)[orig_mask_train.reset_index(drop=True)]
    filtered_trainPredict = pd.Series(trainPredict).reset_index(drop=True)[orig_mask_train.reset_index(drop=True)]

    #filtered_trainY = imputed_train[orig_mask_train]
    #filtered_trainPredict = trainPredict[orig_mask_train]

    filtered_testY = imputed_test.reset_index(drop=True)[orig_mask.reset_index(drop=True)]
    filtered_testPredict = pd.Series(testPredict).reset_index(drop=True)[orig_mask.reset_index(drop=True)]
    # Maskeye göre filtreleme
    #filtered_testY = imputed_test[orig_mask]
    #filtered_testPredict = testPredict[orig_mask]

    # === 7. Performans metrikleri
    def modified_smape(y_true, y_pred):
        return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))
    # === Performans metrikleri

    train_mse = mean_squared_error(filtered_trainY, filtered_trainPredict)
    train_mae = mean_absolute_error(filtered_trainY, filtered_trainPredict)
    train_rmse = np.sqrt(train_mse)
    train_msmape = modified_smape(filtered_trainY, filtered_trainPredict)
    print(train_mse, train_mae, train_rmse, train_msmape) # yüzde olarak ifade etmek için *100

    msmape = modified_smape(filtered_testY, filtered_testPredict)
    mse = mean_squared_error(filtered_testY, filtered_testPredict)
    mae = mean_absolute_error(filtered_testY, filtered_testPredict)
    rmse = np.sqrt(mse)


    print(mse, mae, rmse, msmape)
    print(filtered_testPredict.isna().sum())
    print(filtered_testY.isna().sum())
    summary_records.append({
        "Iteration": run,
        "Imputation Time": imputation_time,
        "Train MSE": train_mse,
        "Train MAE": train_mae,
        "Train RMSE": train_rmse,
        "Train msMAPE": train_msmape,
        "Test MSE": mse,
        "Test MAE": mae,
        "Test RMSE": rmse,
        "Test msMAPE": msmape
    })


# ——— SONUÇLARI KAYDETMEK İÇİN DATAFRAME OLUŞTUR ———
summary_df = pd.DataFrame(summary_records)
summary_df = summary_df.round({
    "Imputation Time": 4,
    "Train MSE": 6, "Train MAE": 6, "Train RMSE": 6, "Train msMAPE": 4,
    "Test MSE": 6, "Test MAE": 6, "Test RMSE": 6, "Test msMAPE": 4
})
summary_df.to_excel("Tepe10.xlsx", index=False)

print("✅ 10 çalıştırmalık özet ve tahmin sonuçları hazır.")


In [ ]:
def create_lagged_features(series, look_back):
    X, y = [], []
    series = np.array(series)
    for i in range(len(series) - look_back):
        X.append(series[i:i + look_back])
        y.append(series[i + look_back])
    return np.array(X), np.array(y)

In [ ]:
#SVR+STACKING

import time
import pandas as pd
import numpy as np
from pykalman import KalmanFilter
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

summary_records = []
prediction_dict = {}

for run in range(1, 11):
    print(f"▶️ Iteration {run} başlıyor...")
    #dataset_path = '/content/drive/MyDrive/eksik_veri_simulasyonlari/MS_missing_with_existing_20pct.xlsx'
    dataset_path = '/content/drive/MyDrive/VNS_dataset/VNS_Tepebasi.xlsx'
    df = pd.read_excel(dataset_path, parse_dates=['Tarih'])
    df.replace("-", np.nan, inplace=True)
    df = df.iloc[18630:]
    print(df.head())
    df['Tarih'] = pd.to_datetime(df['Tarih'], dayfirst=True, errors='coerce')
    df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].apply(lambda x: np.nan if x < 0 else x)
    df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].clip(lower=0, upper=np.nanmax(df['PM10 ( µg/m3 )']))

    series = df['PM10 ( µg/m3 )']
    dates = df['Tarih']
    #series = series.iloc[18630:].reset_index(drop=True)
    #dates = dates.iloc[18630:].reset_index(drop=True)
    print(dates.head())
    n_total = len(series)
    original_nan_mask = series.isna().to_numpy()
    split_idx = int(0.8 * n_total)
    train_series = series[:split_idx].reset_index(drop=True)
    test_series = series[split_idx:].reset_index(drop=True)

    # === 1. IMPUTATION YÖNTEMLERİ ===
    def modified_smape(y_true, y_pred):
      return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))

    def kalman_filter_imputation(train, test):
        kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1,
                          initial_state_covariance=1, transition_matrices=[1],
                          observation_matrices=[1], transition_covariance=1,
                          observation_covariance=1)
        train_masked = np.ma.array(train, mask=np.isnan(train))
        train_imputed, _ = kf.smooth(train_masked)

        combined = np.concatenate([train_imputed.flatten(), test])
        combined_masked = np.ma.array(combined, mask=np.isnan(combined))
        combined_imputed, _ = kf.smooth(combined_masked)
        test_imputed = combined_imputed[len(train):]
        return train_imputed.flatten(), test_imputed.flatten()

    def interpolation_imputation(train, test):
        train_imputed = pd.Series(train).interpolate(limit_direction='both').to_numpy()
        combined = np.concatenate([train_imputed, test])
        combined_imputed = pd.Series(combined).interpolate(limit_direction='both').to_numpy()
        return train_imputed, combined_imputed[len(train):]

    def backward_fill_imputation(train, test):
        return pd.Series(train).bfill().ffill().to_numpy(), pd.Series(test).bfill().ffill().to_numpy()

    def svr_imputation_separate(train, test, n_lags=3):
        def create_lagged_df(series):
            df = pd.DataFrame({'y': series})
            for i in range(1, n_lags + 1):
                df[f'lag_{i}'] = df['y'].shift(i)
            return df

        feature_names = [f'lag_{i}' for i in range(1, n_lags + 1)]
        train_df = create_lagged_df(pd.Series(train)).dropna()
        X_train = train_df[feature_names].to_numpy()
        y_train = train_df['y'].to_numpy()
        svr = SVR().fit(X_train, y_train)

        train_filled = np.copy(train)
        for idx in np.where(np.isnan(train))[0]:
            if idx >= n_lags and not np.isnan(train[idx - n_lags:idx]).any():
                x_pred = train[idx - n_lags:idx][::-1].reshape(1, -1)
                train_filled[idx] = svr.predict(x_pred)[0]
        train_filled = pd.Series(train_filled).interpolate().bfill().ffill().to_numpy()

        test_filled = np.copy(test)
        for idx in np.where(np.isnan(test))[0]:
            full_input = np.concatenate([train_filled[-n_lags:], test[:idx]])
            if len(full_input) >= n_lags and not np.isnan(full_input[-n_lags:]).any():
                x_pred = full_input[-n_lags:][::-1].reshape(1, -1)
                test_filled[idx] = svr.predict(x_pred)[0]
        test_filled = pd.Series(test_filled).interpolate().bfill().ffill().to_numpy()
        return train_filled, test_filled


    start_time = time.time()
    # === 2. TÜM YÖNTEMLERİ UYGULA ===
    kalman_train, kalman_test = kalman_filter_imputation(train_series.to_numpy(), test_series.to_numpy())
    linear_train, linear_test = interpolation_imputation(train_series, test_series)
    bfill_train, bfill_test = backward_fill_imputation(train_series, test_series)
    svr_train, svr_test = svr_imputation_separate(train_series.to_numpy(), test_series.to_numpy())

    # === 3. STACKING VERİSİ (TÜM DOLU GÖZLEMLER) ===
    non_nan_train_idx = np.where(~np.isnan(train_series))[0]

    X_train_stack = np.column_stack([
        kalman_train[non_nan_train_idx],
        linear_train[non_nan_train_idx],
        bfill_train[non_nan_train_idx],
        svr_train[non_nan_train_idx]
    ])
    y_train_stack = train_series.to_numpy()[non_nan_train_idx]

    nan_train_idx = np.where(np.isnan(train_series))[0]
    nan_test_idx = np.where(np.isnan(test_series))[0]

    X_test_stack = np.column_stack([
        kalman_test[nan_test_idx],
        linear_test[nan_test_idx],
        bfill_test[nan_test_idx],
        svr_test[nan_test_idx]
    ])

    # === 4. META MODEL: GRADIENT BOOSTING ===
    if len(X_train_stack) >= 10:
        meta_model = GradientBoostingRegressor()
        meta_model.fit(X_train_stack, y_train_stack)

        # Yalnızca eksik değerler doldurulacak
        train_series.iloc[nan_train_idx] = meta_model.predict(
            np.column_stack([
                kalman_train[nan_train_idx],
                linear_train[nan_train_idx],
                bfill_train[nan_train_idx],
                svr_train[nan_train_idx]
            ])
        )
        test_series.iloc[nan_test_idx] = meta_model.predict(X_test_stack)
    else:
        print("⚠️ Yetersiz eğitim verisi. Kalman ile dolduruluyor.")
        train_series.iloc[nan_train_idx] = kalman_train[nan_train_idx]
        test_series.iloc[nan_test_idx] = kalman_test[nan_test_idx]

    filled_series = pd.concat([train_series, test_series], ignore_index=True)
    end_time = time.time()
    imputation_time = end_time - start_time
    print(f"Imputation Time: {imputation_time} seconds")

    # === 5. GÖRSELLEŞTİRME ===
    plt.figure(figsize=(12, 6))
    plt.plot(dates, series, label='Orijinal Seri (Eksik)', alpha=0.6)
    plt.plot(dates, filled_series, label='Stacking + GB Regressor ile Doldurulmuş Seri')
    plt.xlabel('Tarih')
    plt.ylabel('PM 2.5 ( µg/m3 )')
    plt.title('Eksik Veri Doldurma: Stacking ve GB Regressor')
    plt.legend()
    plt.grid(True)
    plt.show()

    print("Eksik değer sayısı (doldurma sonrası):", filled_series.isnull().sum())
    filled_only = filled_series
    #filled_only = final_filled_series
    print(len(filled_only))

    # === 1. Train / Test böl
    split_ratio = 0.8
    split_idx = int(len(filled_only) * split_ratio)
    train_series = filled_only[:split_idx]
    test_series = filled_only[split_idx:]
    print(len(train_series))
    print(len(test_series))

    print(train_series)
    print(test_series)
    look_back = 24
    X_train, y_train = create_lagged_features(train_series, look_back)
    X_test, y_test = create_lagged_features(test_series, look_back)
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()
    best_params=svr_grid_search(X_train_scaled,y_train_scaled)


    best_svr = SVR(**best_params)
    #best_svr=SVR()
    training_time_start=time.time()
    best_svr.fit(X_train_scaled, y_train_scaled)

    training_time_end=time.time()
    training_time=training_time_end-training_time_start

    # 📌 **7. Tahmin Yapma**
    test_start = time.time()
    y_pred_scaled = best_svr.predict(X_test_scaled)
    test_end = time.time()
    #testPredict = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    # missing value olmayanlardan üzerinden mterik hesaplarken lage göre düzenleme yapmak lazım
    # 📌 **8. Model Performansını Değerlendirme**

    testY = y_test  # y_test zaten mevcut
    trainPredict = scaler_y.inverse_transform(best_svr.predict(X_train_scaled).reshape(-1, 1)).flatten()
    trainY = y_train  # unscaled değer

    testPredict = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    testY = y_test

    orig_train = series[0:split_idx]
    orig_mask_train = ~np.isnan(orig_train)
    valid_mask_train = orig_mask_train[look_back:look_back + len(trainY)]

    print(f"valid_mask_train: {valid_mask_train.shape}")
    print(f"trainY: {trainY.shape}, trainPredict: {trainPredict.shape}")

    # Maskeye göre filtreleme
    trainY = trainY.ravel()
    trainPredict = trainPredict.ravel()
    filtered_trainY = trainY[valid_mask_train]
    filtered_trainPredict = trainPredict[valid_mask_train]


    trainMAE = mean_absolute_error(filtered_trainY, filtered_trainPredict)
    trainMSE = np.sqrt(mean_squared_error(trainY, trainPredict))
    trainmsMAPE = modified_smape(trainY, trainPredict)
    print("train_MAE:", trainMAE)
    print("train_MSE:", trainMSE)
    print("train_modifiedsMAPE:", trainmsMAPE)


    # Orijinal test verisi (eksik değer içeren)
    #orig_test = alldata[train_size+1:]  # Test kısmının orijinali
    #orig_mask = ~np.isnan(orig_test)  # Eksik olmayanlar için maske
    #valid_mask = orig_mask[look_back:]  # look_back sonrası geçerli indeksler

    orig_test = series[split_idx+look_back:]  # Güncellendi
    orig_mask = ~np.isnan(orig_test)
    valid_mask = orig_mask  # look_back sonrası eksik olmayan veriler

    print("TestPredict shape:", testPredict.shape)
    testY = testY.flatten()
    testPredict = testPredict.flatten()

    # Maskeye göre filtreleme
    filtered_testY = testY[valid_mask]
    filtered_testPredict = testPredict[valid_mask]

    print("Filtered TestY shape:", filtered_testY.shape)
    print("Filtered TestPredict shape:", filtered_testPredict.shape)

    if len(filtered_testY) == 0:
        print("Hata: Filtrelenmiş test verileri boş.")
    else:
        testMAE = mean_absolute_error(filtered_testY, filtered_testPredict)
        testMSE = np.sqrt(mean_squared_error(filtered_testY, filtered_testPredict))
        testmsMAPE = modified_smape(filtered_testY, filtered_testPredict)
        print("test_MAE:", testMAE)
        print("test_MSE:", testMSE)
        print("test_modifiedsMAPE:", testmsMAPE)

    '''
    # === 8. Grafik
    plt.figure(figsize=(15, 5))
    plt.plot(y_true_filtered, label="Gerçek", linewidth=2)
    plt.plot(y_pred_filtered, label="Tahmin", linewidth=2, alpha=0.7)
    plt.title("LSTM Test Tahmin Performansı")
    plt.legend()
    plt.grid(True)
    plt.show()
    '''
    summary_records.append({
        "Iteration": run,
        "Imputation Time": imputation_time,
        "Train MAE": trainMAE,
        "Train RMSE": trainMSE,
        "Train msMAPE": trainmsMAPE,
        "Test MAE": testMAE,
        "Test RMSE": testMSE,
        "Test msMAPE": testmsMAPE
    })

    # Tahmin değerlerini sakla (her iterasyon için ayrı sütun)
    prediction_dict[f"y_true_{run}"] = filtered_testY
    prediction_dict[f"y_pred_{run}"] = filtered_testPredict

# ——— SONUÇLARI KAYDETMEK İÇİN DATAFRAME OLUŞTUR ———
summary_df = pd.DataFrame(summary_records)
summary_df = summary_df.round({
    "Imputation Time": 4,
    "Train MSE": 6, "Train MAE": 6, "Train RMSE": 6, "Train msMAPE": 4,
    "Test MSE": 6, "Test MAE": 6, "Test RMSE": 6, "Test msMAPE": 4
})
summary_df.to_excel("MS20_imputation_svr_summary.xlsx", index=False)


# Tüm tahminleri hizalamak için en kısa uzunluğu belirle
min_len = min(len(v) for v in prediction_dict.values())
prediction_df = pd.DataFrame({k: v[:min_len] for k, v in prediction_dict.items()})

print(prediction_df.head())
# Excel’e kaydet (çalıştırmak istersen aşağıdaki yorumları kaldır)
# summary_df.to_excel("imputation_lstm_summary.xlsx", index=False)
prediction_df.to_excel("MS20_imputation_svr_predictions.xlsx", index=False)

